This code is also in standalone python scripts. Based on a Google Plus article on getting data from webscaping led to interesting playing with selenium and beautifulsoup. There's another set of code for volcanoes that I'd like to bump up to do some playing with scikit-learn and maybe TensorFlow.
Here, create two functions, get_all_earthquakes and getEarthquakeDetails.

In [1]:
import pandas as pd
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
def get_all_earthquakes(html_soup):
    earthquakes = html_soup.findAll("li",attrs={'class':'eqitem'})
    earthquakedata = []
    #sys.stderr.write(repr(html_soup)+'\n')
    for earthquake in earthquakes:
        title = earthquake.find(attrs={'class':'title'})
        data = earthquake.findAll('span')
        row_cells = earthquake.findAll("span")
        earthquake_entry = {
            "name":title.text,
            "usgsref":title['href'],
            "strength":row_cells[0].text.strip(),
            "datetime":row_cells[1].text.strip(),
            "depthKM":row_cells[2].text.strip().split()[0]
        }
        earthquakedata.append(earthquake_entry)
    return earthquakedata

def getEarthquakeDetails(soup):
    details = {}
    # the usgs page has a *lot* more data than what I'm grabbing now.
    evheader = soup.find('header',attrs={'class':'event-header'})
    location = evheader.find('span',attrs={'class':'event-coordinates'})
    details['location'] = location.text
    return details


Equivalent to the 'main'. Makes use of selenium package which connects as a Chrome browser to usgs earthquake data.
Then pass parsed html to get_all_earthquakes, build dataframe out of it,
get more details and build a more detailed dataframe.

In [3]:
options = webdriver.ChromeOptions()
options.set_headless(True)
browser = webdriver.Chrome(chrome_options=options)
browser.get('https://earthquake.usgs.gov/earthquakes/browse/m6-world.php?year=2001')
html = browser.page_source
html_soup = BeautifulSoup(html, 'html.parser')
earthquake_list = get_all_earthquakes(html_soup)
earthquakedf = pd.DataFrame(earthquake_list)
# This part is really slow. Probably more sense to use the fdsnws
# catalog with a dedicated api (https://earthquake.usgs.gov/fdsnws/).
for earthquake in earthquake_list:
    browser.get(earthquake['usgsref'])
    detailSoup = BeautifulSoup(browser.page_source, 'html.parser')
    earthquake.update(getEarthquakeDetails(detailSoup))
earthquakedf = pd.DataFrame(earthquake_list)
sys.stderr.write(repr(earthquakedf.head(5)) +'\n')


                  datetime depthKM                                  location  \
0  2001-12-28 22:09:28 UTC   160.6    \n        8.357°S     \n    74.220°W     
1  2001-12-27 10:54:51 UTC   153.2  \n        14.647°S     \n    167.262°E     
2  2001-12-23 22:52:54 UTC    16.0   \n        9.613°S     \n    159.530°E     
3  2001-12-22 00:40:04 UTC    74.2  \n        10.910°S     \n    165.863°E     
4  2001-12-18 04:02:58 UTC    14.0  \n        23.954°N     \n    122.734°E     

                 name strength  \
0        central Peru      6.0   
1             Vanuatu      6.2   
2     Solomon Islands      6.8   
3  Santa Cruz Islands      6.0   
4       Taiwan region      6.8   

                                             usgsref  
0  https://earthquake.usgs.gov/earthquakes/eventp...  
1  https://earthquake.usgs.gov/earthquakes/eventp...  
2  https://earthquake.usgs.gov/earthquakes/eventp...  
3  https://earthquake.usgs.gov/earthquakes/eventp...  
4  https://earthquake.usgs.gov/earthqua